In [2]:
import pandas as pd
import numpy as np
import re
wholetext=pd.read_csv("/Users/empathybroker/Documents/Synonyms/cbn-data-experiment-synonyms/zara.csv",sep=",",header=0)

In [3]:
#CLEANING DATA
zaradf=wholetext.dropna(axis=0,how="any") #get rid of nan values
zaradf1nan=zaradf[~zaradf[zaradf.columns[0]].str.contains("\d")] #get rid of number values
zaradf1nan=zaradf1nan[~zaradf1nan[zaradf1nan.columns[1]].str.contains("\d")] #all numbers removed

In [4]:
#this to extract the most important product numbers, this gives me a df with most common Pnum as an index
wzara=zaradf1nan.groupby(zaradf1nan.columns[2]).sum() 
szara=wzara.sort_values(by=wzara.columns[1],ascending=False) 
maxPnum=zaradf1nan.groupby(zaradf1nan.columns[2]).sum().sort_values(by=wzara.columns[1],ascending=False)
#by setting Pnum as a column to be able to extract the numbers as a list
maxPnum.reset_index(level=0, inplace=True)#to set Pnum as a column as it was an index

In [5]:
#this to drop useless columns
zaradf1nan.drop(zaradf1nan.columns[3],axis=1,inplace=True)
zaradf1nan.drop(zaradf1nan.columns[3],axis=1,inplace=True)

In [6]:
#now making Pnum the index for ease of loaction
zaradf1nan.set_index(zaradf1nan.columns[2],inplace=True)

In [13]:
def no_short_words(list):
    return (j for j in list if len(j)>2) #this function to get rid of small words
from collections import Counter #this is needed to count words and expressions without CountVectorizer

counter=maxPnum[maxPnum.columns[0]]#.head(100)#this head() to read the 100 most clicked on products
newtags=[]
colnames=("prod_id","(searches, freq)","(top_words, freq)")
newtagsdf1=pd.DataFrame(columns=colnames)
contador=[]
for i in counter:
    zaradfnum=zaradf1nan.loc[[i]]
    fsearch=zaradfnum["fsearch"].astype(str).values.tolist()
    fsearch=[re.sub(r"[^a-zA-Z ]","",elem) for elem in fsearch] # a-z characters only and spaces
    cleansearch=no_short_words(fsearch)
    counts = Counter(cleansearch)
    topcand=counts.most_common(5)
    tempdf=pd.DataFrame(columns=colnames)
    wordlist=[]
    totsearch=[]
    for search in fsearch: #this is to make a single list and be able to count most common words
        words=search.split()
        totsearch.append(words)
    for sublist in totsearch:
        for word in sublist:
            wordlist.append(word)
    tempdf["prod_id"]=zaradfnum.index[:1]
    tempdf["(searches, freq)"]=[topcand]
    tempdf["(top_words, freq)"]=[Counter(no_short_words(wordlist)).most_common(5)]
    newtagsdf1=newtagsdf1.append(tempdf,ignore_index=True)

KeyboardInterrupt: 

In [7]:
newtagsdf1.to_csv("counter100.csv",sep="\t")

In [12]:
newtagsdf1.loc[[4238059]]

KeyError: 'None of [[4238059]] are in the [index]'

In [10]:
newtagsdf1.prod_id=newtagsdf1.prod_id.astype(int)

In [1]:
newtagsdf1.loc[[4238059]]

NameError: name 'newtagsdf1' is not defined

In [9]:
writer = pd.ExcelWriter('no_result_searches.xlsx')
newtagsdf1.to_excel(writer,'Sheet1')
writer.save()